In [242]:
import pandas as pd

In [243]:
pd.options.display.max_columns = None

In [244]:
column_names = ['user_id', 'id', 'count_months', 'amount', 'month_payment',
       'payed_this_month', 'to_pay', 'planned', 'created_at', 'date']
train = pd.DataFrame(columns=column_names)

In [245]:
train.head()

,user_id,id,count_months,amount,month_payment,payed_this_month,to_pay,planned,created_at,date


In [246]:
import mysql.connector
conn = mysql.connector.connect(host='localhost', 
                port=3306, user='root', passwd='dalekini', 
                db='digiu')
jan = pd.read_sql_query("SELECT distinct cr.user_id as user_id, cr.id as id, cr.count_months as count_months, cr.amount as amount, cr.month_payment as month_payment,inv.amount as payed_this_month, cps.left_amount as to_pay, cps.amount as planned, cr.created_at as created_at, cps.date as date FROM credit cr join credit_payment_schedule cps on cr.id = cps.credit_id left join (select credit_id, sum(amount)as amount from invoice where created_at > '2020-01-01' AND created_at < '2020-02-01' group by credit_id) inv on cr.id=inv.credit_id WHERE (cr.created_at > '2019-12-01' AND cr.created_at < '2020-01-01') and (cps.date > '2020-01-01' AND cps.date < '2020-02-01') and cr.status in (2) and (cr.user_id > 19) and cr.user_id not in (23,24,26,28,33)", conn)
train = train.append(jan)

In [247]:
train.head()

,user_id,id,count_months,amount,month_payment,payed_this_month,to_pay,planned,created_at,date
0,196,6,6,1000.0,100.0,100.0,0.0,100.0,2019-12-18 23:55:33,2020-01-18
1,227,20,7,500.0,50.0,50.0,0.0,50.0,2019-12-20 05:40:39,2020-01-19
2,238,22,7,500.0,50.0,50.0,0.0,50.0,2019-12-20 10:45:33,2020-01-20
3,45,24,26,5000.0,167.0,167.0,0.0,167.0,2019-12-20 14:59:29,2020-01-20
4,123,26,28,5000.0,167.0,167.0,0.0,167.0,2019-12-20 16:56:37,2020-01-20


In [248]:
train.shape

(147, 10)

In [249]:
feb = pd.read_sql_query("SELECT distinct cr.user_id as user_id, cr.id as id, cr.count_months as count_months, cr.amount as amount, cr.month_payment as month_payment,inv.amount as payed_this_month, cps.left_amount as to_pay, cps.amount as planned, cr.created_at as created_at, cps.date as date FROM credit cr join credit_payment_schedule cps on cr.id = cps.credit_id left join (select credit_id, sum(amount)as amount from invoice where created_at > '2020-02-01' AND created_at < '2020-03-01' group by credit_id) inv on cr.id=inv.credit_id WHERE (cr.created_at > '2019-12-01' AND cr.created_at < '2020-02-01') and (cps.date > '2020-02-01' AND cps.date < '2020-03-01') and cr.status in (2) and (cr.user_id > 19) and cr.user_id not in (23,24,26,28,33)", conn)
train = train.append(feb)

In [250]:
train.shape

(366, 10)

In [251]:
mar = pd.read_sql_query("SELECT distinct cr.user_id as user_id, cr.id as id, cr.count_months as count_months, cr.amount as amount, cr.month_payment as month_payment,inv.amount as payed_this_month, cps.left_amount as to_pay, cps.amount as planned, cr.created_at as created_at, cps.date as date FROM credit cr join credit_payment_schedule cps on cr.id = cps.credit_id left join (select credit_id, sum(amount)as amount from invoice where created_at > '2020-03-01' AND created_at < '2020-04-01' group by credit_id) inv on cr.id=inv.credit_id WHERE (cr.created_at > '2019-12-01' AND cr.created_at < '2020-03-01') and (cps.date > '2020-03-01' AND cps.date < '2020-04-01') and cr.status in (2) and (cr.user_id > 19) and cr.user_id not in (23,24,26,28,33)", conn)
train = train.append(mar)

In [252]:
train.shape

(1087, 10)

In [253]:
train = train.fillna(0)

In [254]:
test = pd.read_sql_query("SELECT distinct cr.user_id as user_id, cr.id as id, cr.count_months as count_months, cr.amount as amount, cr.month_payment as month_payment,inv.amount as payed_this_month, cps.left_amount as to_pay, cps.amount as planned, cr.created_at as created_at, cps.date as date FROM credit cr join credit_payment_schedule cps on cr.id = cps.credit_id left join (select credit_id, sum(amount)as amount from invoice where created_at > '2020-04-01' AND created_at < '2020-05-01' group by credit_id) inv on cr.id=inv.credit_id WHERE (cr.created_at > '2019-12-01' AND cr.created_at < '2020-04-01') and (cps.date > '2020-04-01' AND cps.date < '2020-05-01') and cr.status in (2) and (cr.user_id > 19) and cr.user_id not in (23,24,26,28,33)", conn)

In [255]:
test.shape

(1038, 10)

In [256]:
test = test.fillna(0)

In [257]:
left_train = sum((train.to_pay))
print(left_train)

17092.63


In [258]:
Actually_payed_train = sum(train.payed_this_month)
print(Actually_payed_train)

39889.34


In [259]:
Planned_train = sum(train.planned)
print(Planned_train)

57482.0


In [260]:
Delta_train = Planned_train - left_train - Actually_payed_train
print(Delta_train)

500.03


In [261]:
Planned_in_april = sum(test.planned)
print(Planned_in_april)

46885.0


In [262]:
Actually_payed_april = sum(test.payed_this_month)
print(Actually_payed_april)

1010.29


In [263]:
left_april = sum((test.to_pay).astype(float))
print(left_april)

36538.5


In [264]:
Delta_test = Planned_in_april - left_april - Actually_payed_april
print(Delta_test)

9336.21


In [265]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, r2_score
from catboost import CatBoostRegressor
import seaborn as sns
import matplotlib.pyplot as plt
features = ['count_months', 'amount', 'month_payment']

X_train = train[features]
X_val = test[features]
y_train = train.payed_this_month.astype('int64').copy()
y_val = test.payed_this_month.astype('int64').copy()

model=CatBoostRegressor(iterations=200, depth=5, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train, y_train)
preds = model.predict(X_val)
print(sum(preds))
pred_catboost = sum(preds)

0:	learn: 51.3830717	total: 1.48ms	remaining: 294ms
1:	learn: 49.3532694	total: 2.6ms	remaining: 257ms
2:	learn: 47.7854996	total: 3.42ms	remaining: 225ms
3:	learn: 46.4469117	total: 4.37ms	remaining: 214ms
4:	learn: 45.3125990	total: 5.39ms	remaining: 210ms
5:	learn: 44.4158603	total: 6.8ms	remaining: 220ms
6:	learn: 43.3820388	total: 9.12ms	remaining: 251ms
7:	learn: 42.6447569	total: 10.1ms	remaining: 243ms
8:	learn: 41.8177251	total: 11.2ms	remaining: 237ms
9:	learn: 41.2408823	total: 11.9ms	remaining: 226ms
10:	learn: 40.7606379	total: 13ms	remaining: 223ms
11:	learn: 40.5427621	total: 13.9ms	remaining: 217ms
12:	learn: 40.0972578	total: 14.3ms	remaining: 206ms
13:	learn: 39.6575337	total: 14.8ms	remaining: 196ms
14:	learn: 39.1736538	total: 15.2ms	remaining: 187ms
15:	learn: 38.8526381	total: 15.6ms	remaining: 180ms
16:	learn: 38.5382312	total: 16.1ms	remaining: 173ms
17:	learn: 38.2242509	total: 16.9ms	remaining: 171ms
18:	learn: 37.8898887	total: 18.9ms	remaining: 180ms
19:	lea

In [266]:
clf = ensemble.RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

print(sum(y_pred))
pred_randforest = sum(y_pred)

24406


In [267]:
#Предскажем деревом решений

In [268]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x,y)
proba = model.predict(x_val)

print( sum(proba))
pred_destree = sum(proba)

23444


In [269]:
prediction = (pred_catboost+pred_destree+pred_randforest)/3
print(prediction)

24377.08185137349
